<h1 align="center"><font size="8"> scVelo Analysis </font></h1>

> Huitian (Yolanda) Diao, Sept. 14 2020 <br>
> Source: [Velocyto notebook](https://github.com/velocyto-team/velocyto-notebooks.git)


- vlm object:
    - Gene IDs: vlm.ra['Gene']
    - Cell IDs: vlm.ca['CellID']

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import copy
import sys
from importlib import reload
from pathlib import Path

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import scvelo as scv
scv.logging.print_version()


scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.set_figure_params('scvelo')  # for beautified visualization

%matplotlib inline

Running scvelo 0.2.2 (python 3.8.3) on 2020-11-26 12:17.


In [2]:
code_dir = os.getcwd()
base_dir = code_dir.replace("/codes_local","")
parent_dir = "/".join(base_dir.split("/")[:-1])
tools_dir = parent_dir + "/tools"

sys.path.insert(1, tools_dir)
import sc_addons
reload(sc_addons)

sp_name = base_dir.split("/")[-1]
analysis_name = ["0", "resampled"]

wk_dir = base_dir + "/2_scVelo/%s_scVelo_out_%s"%(analysis_name[0],analysis_name[1])
Path(wk_dir).mkdir(parents=True, exist_ok=True)
os.chdir(wk_dir)

print(sp_name, analysis_name)

0_Acute-Chronic ['0', 'resampled']


In [3]:
de_dir = wk_dir + '/2_DE'
Path(de_dir).mkdir(parents=True, exist_ok=True)

## Read input

In [7]:
adata = scv.read('%s_scVelo_postPT.h5ad'%sp_name, compression='gzip')
adata.obs['cell_type_general'] = [x[0] for x in adata.obs['cell_type']]
adata.obs['cell_type_general'] = adata.obs['cell_type_general'].astype('category')
adata_act = adata[adata.obs['cell_type_general'] != "N"]

gene_n = len(adata.var)

### Per Cluster Arm v.s. Cl13

*Velocity*

In [64]:
louvain_unique = np.unique(adata.obs['louvain'])
for i in louvain_unique:
    adata_i = adata_act.copy()
    adata_i = adata_i[adata_i.obs['louvain'] == i]
    
    types_sum_i = adata_i.obs.groupby('cell_type_general').count()
    types_count_i = list(types_sum_i.iloc[:,1])
    if  len(types_count_i) > 1 and min(types_count_i) > 10:
        scv.tl.rank_velocity_genes(adata_i, n_genes=genes_n, groupby="cell_type_general")
        gn_df = pd.DataFrame(adata_i.uns['rank_velocity_genes']['names'])
        score_df = pd.DataFrame(adata_i.uns['rank_velocity_genes']['scores'])

        a_df = pd.DataFrame({'gene_names': gn_df['A'].tolist(), 'scores': score_df['A'].tolist()})
        c_df = pd.DataFrame({'gene_names': gn_df['C'].tolist(), 'scores': score_df['C'].tolist()})

        a_df.to_csv("%s/%s_Arm-vs-Cl13_diffVelocity.csv"%(de_dir, i), index=False)
        c_df.to_csv("%s/%s_Cl13-vs-Arm_diffVelocity.csv"%(de_dir, i), index=False)

ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:02) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:02) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


...

Trying to set attribute `.var` of view, copying.


    finished (0:00:01) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:01) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)


KeyError: 'C'

*Unspliced*

In [41]:
louvain_unique = np.unique(adata.obs['louvain'])
for i in louvain_unique:
    adata_i = adata_act.copy()
    adata_i = adata_i[adata_i.obs['louvain'] == i]
    
    types_sum_i = adata_i.obs.groupby('cell_type_general').count()
    types_count_i = list(types_sum_i.iloc[:,1])
    if  len(types_count_i) > 1 and min(types_count_i) > 10:
        scv.tl.rank_velocity_genes(adata_i, n_genes=gene_n, groupby="cell_type_general", vkey='Mu')
        gn_df = pd.DataFrame(adata_i.uns['rank_velocity_genes']['names'])
        score_df = pd.DataFrame(adata_i.uns['rank_velocity_genes']['scores'])

        a_df = pd.DataFrame({'gene_names': gn_df['A'].tolist(), 'scores': score_df['A'].tolist()})
        c_df = pd.DataFrame({'gene_names': gn_df['C'].tolist(), 'scores': score_df['C'].tolist()})

        a_df.to_csv("%s/%s_Arm-vs-Cl13_diffMu.csv"%(de_dir, i), index=False)
        c_df.to_csv("%s/%s_Cl13-vs-Arm_diffMu.csv"%(de_dir, i), index=False)

ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:04) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:04) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


...

    finished (0:00:03) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:03) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:02) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)


*Spliced*

In [42]:
louvain_unique = np.unique(adata.obs['louvain'])
for i in louvain_unique:
    adata_i = adata_act.copy()
    adata_i = adata_i[adata_i.obs['louvain'] == i]
    
    types_sum_i = adata_i.obs.groupby('cell_type_general').count()
    types_count_i = list(types_sum_i.iloc[:,1])
    if  len(types_count_i) > 1 and min(types_count_i) > 10:
        scv.tl.rank_velocity_genes(adata_i, n_genes=gene_n, groupby="cell_type_general", vkey='Ms')
        gn_df = pd.DataFrame(adata_i.uns['rank_velocity_genes']['names'])
        score_df = pd.DataFrame(adata_i.uns['rank_velocity_genes']['scores'])

        a_df = pd.DataFrame({'gene_names': gn_df['A'].tolist(), 'scores': score_df['A'].tolist()})
        c_df = pd.DataFrame({'gene_names': gn_df['C'].tolist(), 'scores': score_df['C'].tolist()})

        a_df.to_csv("%s/%s_Arm-vs-Cl13_diffMs.csv"%(de_dir, i), index=False)
        c_df.to_csv("%s/%s_Cl13-vs-Arm_diffMs.csv"%(de_dir, i), index=False)

ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:03) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:02) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


...

    finished (0:00:02) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:01) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
ranking velocity genes


Trying to set attribute `.var` of view, copying.


    finished (0:00:04) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)
